In [1]:
import openai

f = open('../api_key.txt')
api_key = f.read()
openai.api_key = api_key

In [2]:
import json

# 날씨를 물어보면 같은 답을 return하는 dummy function 생성.
def get_current_weather(location, unit='fahrenheight'):
    weather_info = {
        'location': location,
        'temperature': '72',
        'unit': unit,
        'forecast': ['sunny', 'windy']
    }
    # 실제 날씨를 불러오는 API를 사용해 그 값을 return하는 코드를 사용한다면 실행할 때마다 날씨를 알 수 있을 것
    return json.dumps(weather_info)

functions = [
    {
        'name': 'get_current_weather', # function의 이름
        'description': 'Get the current weather in a given location', # 해당 function의 설명
        'parameters': { 
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': 'The city and state, e.g. San Francisco, CA' # 파라미터의 속성 설명
                },
                'unit': {
                    'type': 'string',
                    'enum': ['celsius', 'fahrenheit']
                }
            },
            'required': ['location']
        }
    }
]

# 작성한 메시지, 설명을 담은 functions로 API 호출
# fucntion_call을 auto로 설정해 메시지의 의도를 파악한 모델이 function을 사용해야할지 판단
# 보스턴의 날씨를 알려달라는 메시지의 내용, 특정 지역의 날씨를 얻는 function의 내용을 모델이 파악
# function을 사용해야 함을 이해하고 어떤 fucntion인지(name), 어떤 argument가 필요한지 제공
# 그 정보를 response
# 해당 정보를 이용해 다음 api request에 적용해 응답을 기대할 수 있다.
messages = [{'role': 'user', 'content': "What's the weather like in Boston?"}]
response = openai.ChatCompletion.create(
    model='gpt-4-0613',
    messages=messages,
    functions=functions,
    function_call='auto'
)
response_message = response['choices'][0]['message']
print(response_message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}


In [3]:
# 반드시 모든 request가 function을 필요로할 것은 아니므로 function_call 유무로 funtion 사용이 필요한지 파악
# 만약 function_call이 없다면 일반적인 content가 response됐을 것이므로 else는 pass, 이후의 코드를 자유롭게 작성하면 될듯하다
if response_message.get('function_call'):
    available_fuctions = { # 사용자가 만든 function의 목록
        'get_current_weather': get_current_weather
    }

    # api response로 받은 데이터 중 function name 얻기
    function_name = response_message['function_call']['name']

    # 사용자의 function 중 api가 필요로 하는 function 가져오기
    function_to_call = available_fuctions[function_name]

    # 이전 request의 messages에서 모델이 얻은 arguments를 가져오기
    function_args = json.loads(response_message['function_call']['arguments'])

    # print(function_args)

    # 사용자의 function 실행, return값 얻기
    function_response = function_to_call(
        location=function_args.get('location'),
        unit=function_args.get('unit')
    )
    
    print(function_response)


{"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}


In [4]:
# 이전 fucntion_call response를 메시지에 추가
# !위 셀의 if문 블록 내에서 실행되는 것이라 생각해야 함
messages.append(response_message)

# 앞서 구한 function name, function의 return값을 각 필드의 속성으로 가지는 function role의 메시지 추가
messages.append(
    {
        'role': 'function',
        'name': function_name,
        'content': function_response
    }
)

# request할 message는 다음과 같이
# 1.유저의 prompt, 2.이전 response로 얻은 function_call, 3.function의 이름 및 내용
print(messages)

[{'role': 'user', 'content': "What's the weather like in Boston?"}, <OpenAIObject at 0x246ca075450> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}, {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Boston, MA", "temperature": "72", "unit": null, "forecast": ["sunny", "windy"]}'}]


In [5]:
second_response = openai.ChatCompletion.create(
    model='gpt-4-0613',
    messages=messages
)
print('최종 response message:', second_response['choices'][0]['message'])

최종 response message: {
  "role": "assistant",
  "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees."
}


In [8]:
basic_response = openai.ChatCompletion.create(
    model='gpt-4-0613',
    messages=[
        {'role': 'user', 'content': "What's the weather like in Boston?"}
    ]
)
print('일반적인 response:', basic_response['choices'][0]['message']['content'])
print('함수 활용 response:', second_response['choices'][0]['message']['content'])

일반적인 response: Sorry, as an artificial intelligence, I don't have real-time capabilities to provide current weather updates.
함수 활용 response: The weather in Boston is currently sunny and windy with a temperature of 72 degrees.
